#### LLM Powered Chatbot

In [5]:

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("QROQ_API_KEY")

In [7]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "Gemma2-9b-It", groq_api_key = groq_api_key)
model.invoke("Hey").content

'Hey there! 👋 What can I do for you today? 😊\n'

In [12]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content = "Hey, my name is Jim and I am a Generative AI Engineer")]).content

"Hi Jim, it's nice to meet you!\n\nThat's a fascinating field to be in. What kind of generative AI work are you involved with?  \n\nDo you have any projects you're particularly excited about?\n"

In [14]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content = "Hey, my name is Jim and I am a Generative AI Engineer"),
        AIMessage(content = "Hi Jim, it's nice to meet you!\n\nThat's a fascinating field to be in. What kind of generative AI work are you involved with?  \n\nDo you have any projects you're particularly excited about?\n"),
        HumanMessage(content = "What is my name and what do I do ?")
    ]
).content

"You told me your name is Jim and that you are a Generative AI Engineer.  \n\nIs there anything else you'd like to tell me about yourself? 😊  \n"

### Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.

In [18]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [19]:
config = {"configurable": {"session_id": "chat1"}}

In [21]:
response = with_message_history.invoke(
  [HumanMessage(content="Hey my name is Jim and I am an AI Enginner.")],
  config=config
 )

In [22]:
response.content

"Hi Jim! It's great to meet another AI engineer.  \n\nWhat brings you here today? Are you working on any interesting projects?  Perhaps I could be of assistance? \n\n"

In [25]:
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
).content

"You said your name is Jim when you first introduced yourself. 😊 \n\nIs there something specific you'd like to know about your name? For example, are you curious about its origin or meaning?  \n\n\n\n\n\n"

In [27]:
#chamging the config
config_1 = {"configurable": {"session_id": "chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content = "What is my name")],
    config=config_1
)
print(response.content)

As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'm happy to use it!



In [28]:
response = with_message_history.invoke(
    [HumanMessage(content = "My name is jimmy")],
    config=config_1
)
print(response.content)

Hi Jimmy! It's nice to meet you. 😊

Is there anything I can help you with today?



In [29]:
response = with_message_history.invoke(
    [HumanMessage(content = "What is my name")],
    config=config_1
)
print(response.content)

Your name is Jimmy!  

I remember you told me a moment ago. 😊  Anything else I can help you with, Jimmy?

